In [12]:
lammps_input_script = """# Sample LAMMPS input script for viscosity of 2d LJ liquid
# NEMD via fix deform and fix nvt/sllod

# Settings
variable	    x equal 20
variable	    y equal 20

variable	    rho equal 0.6
variable        t equal 1.0
variable	    rc equal 2.5

variable	    srate equal 2.7 # velocity of top edge

# Problem setup
units		    lj
dimension	    2
atom_style	    atomic
neigh_modify	delay 0 every 1

lattice         sq2 ${rho}
region          simbox prism 0 $x 0 $y -0.1 0.1 0.0 0.0 0.0
create_box      1 simbox
create_atoms    1 box

pair_style      lj/cut ${rc}
pair_coeff      * * 1 1

mass            * 1.0

# Equilibration run
velocity        all create $t 97287
fix             1 all nve
fix	            2 all langevin $t $t 0.1 498094
fix	            3 all enforce2d

thermo          1000
run	            5000

unfix  	        1
unfix		    2

# Turn on NEMD shear and equilibrate some more
velocity	    all scale $t

# Shear rate defined relative to perpendicular dimension
variable	    xyrate equal ${srate}/ly 

fix		        1 all nvt/sllod temp $t $t 0.1

# Perform deformation every 1 timestep
fix		        2 all deform 1 xy erate ${xyrate} remap v

# The average in each chunk is computed every 5000 timesteps using 250 samples taken at intervals of 
# 20 timesteps from the preceding timesteps.
# I think center means that coord1 is the center of the bin
compute         layers all chunk/atom bin/1d y upper 0.05 units reduced
fix		        4 all ave/chunk 20 250 5000 layers vx file profile.nemd.2d

compute		    usual all temp
compute		    tilt all temp/deform

thermo          1000
thermo_style	custom step temp c_usual epair etotal press pxy
thermo_modify	temp tilt
run		        50000

# Data gathering run
variable	    visc equal -pxy/(v_srate/ly)
fix		        vave all ave/time 10 100 1000 v_visc ave running start 56000

thermo_style	custom step temp press pxy v_visc f_vave
thermo_modify	temp tilt

# only need to run for 5400 steps to make a good 100-frame movie
# set 54K steps above in equil and 5400 here simply to make good movie
# 54K and 5400 are multiples of box-swap periodicity = 2700 steps

dump	        1 all custom 50 dump.nemd.2d id type x y z vx

run		        50000
"""

In [13]:
# Write the content to a file called in.nemd.2d
with open("in.nemd.2d", "w") as file:
    file.write(lammps_input_script)

In [ ]:
# Continue below!
# Want each timestep to be a dictionary key

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Initialize dictionaries to store the data for each chunk
chunk_data = {}

# Read the file and parse the data
with open('profile_center.nemd.2d', 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith('#') or line.strip() == '':
            continue
        parts = line.split()
        if len(parts) == 3:
            # This is a timestep line
            current_timestep = int(parts[0])
        elif len(parts) == 4:
            # This is a data line
            chunk = int(parts[0])
            coord = float(parts[1])
            ncount = float(parts[2])
            vx = float(parts[3])
            
            if current_timestep not in chunk_data:
                chunk_data[current_timestep] = {'chunk': [], 'coord1': [], 'ncounts': [], 'vx': []}
                
            chunk_data[current_timestep]['chunk'].append(chunk)
            chunk_data[current_timestep]['coord1'].append(coord)
            chunk_data[current_timestep]['ncounts'].append(ncount)
            chunk_data[current_timestep]['vx'].append(vx)

# Function to plot data for a specific timestep using Plotly
def plot_timestep(timestep):
    if timestep in chunk_data:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=chunk_data[timestep]['vx'],
            y=chunk_data[timestep]['coord1'],
            mode='lines+markers',
            name=f'Timestep {timestep}'
        ))
        fig.update_layout(
            title=f'Velocity Profile at Timestep {timestep}',
            xaxis_title='Coord1',
            yaxis_title='vx',
            template='plotly_white'
        )
        fig.show()
    else:
        print(f"No data available for timestep {timestep}")

In [30]:
plot_timestep(10000)